In [2]:
# Import de librairies

import pandas as pd
import numpy as np
import os
CP = "9920510N"

In [3]:
# pre-process ref gare
path_ref = r"C:\Users\{CP}\SNCF\Pôle data - Lab' MTA GrpO365 - Documents\Projets\Hackathon\Données\Données brutes\PA_UIC.csv".format(CP=CP)
data_ref = pd.read_csv(path_ref, encoding='latin-1',sep=';')
display(data_ref)

,PointArret,LibelleLong,UIC,CodeGare
0,62,Cergy le Haut,382655.0,CJV
1,138,Maisons-Laffitte,386425.0,MLF
2,512,Bourg-la-Reine,758698.0,BQQ
3,528,Sceaux,758706.0,SKX
4,529,Fontenay-aux-Roses,758714.0,FNR
...,...,...,...,...
977,46,NÃ©zel - Aulnay,381731.0,NZL
978,524,La HacquiniÃ¨re,758862.0,HAQ
979,525,Gif-sur-Yvette,758870.0,GIF
980,526,Courcelle sur Yvette,758888.0,CVW


In [4]:
# Pre-processing Validations

path_val = r"C:\Users\{CP}\SNCF\Pôle data - Lab' MTA GrpO365 - Documents\Projets\Hackathon\Données\Données brutes\Validations_Ligne_H_12_2022_12_2023.csv".format(CP=CP)
data_val = pd.read_csv(path_val, encoding='latin-1')

table_PA_XXX = dict(zip(data_ref['PointArret'],data_ref['CodeGare']))
data_val['CodeGare'] = data_val['PA'].map(table_PA_XXX)
data_val_final = data_val.drop(['ventilation_ligne','validations_entree','validations_sortie','PA','type_jour','Key'],axis=1).rename(columns={'date':'Date','CodeGare':'Gare_A'})
data_val_final['Date'] = data_val_final['Date'].apply(lambda x: str(x)[:10])
data_val_final['Datetime'] = data_val_final['Date'] + ' ' + data_val_final['tranche_horaire'].replace({24:0,25:1,26:2}).astype(str) +':'+ data_val_final['demi_heure'].apply(lambda x: '00' if x=='0-30' else '30')+":00"

data_val_final['Datetime'] = pd.to_datetime(data_val_final['Datetime'])
display(data_val_final[data_val_final.Date=="2023-11-01"])

,Ligne,Date,gare,validations_entree_brutes,tranche_horaire,demi_heure,validations_sortie_brutes,Gare_A,Datetime
14756,H,2023-11-01,PARIS NORD,363,24,0-30,440,GDS,2023-11-01 00:00:00
14757,H,2023-11-01,PARIS NORD,241,24,30-60,238,GDS,2023-11-01 00:30:00
14758,H,2023-11-01,PARIS NORD,48,25,0-30,36,GDS,2023-11-01 01:00:00
14759,H,2023-11-01,PARIS NORD,14,25,30-60,5,GDS,2023-11-01 01:30:00
14760,H,2023-11-01,PARIS NORD,11,4,0-30,3,GDS,2023-11-01 04:00:00
...,...,...,...,...,...,...,...,...,...
551466,H,2023-11-01,ST OUEN LIESSE,3,21,30-60,0,SOL,2023-11-01 21:30:00
551467,H,2023-11-01,ST OUEN LIESSE,4,22,0-30,0,SOL,2023-11-01 22:00:00
551468,H,2023-11-01,ST OUEN LIESSE,1,22,30-60,0,SOL,2023-11-01 22:30:00
551469,H,2023-11-01,ST OUEN LIESSE,4,23,0-30,0,SOL,2023-11-01 23:00:00


In [5]:
# Pre-processing PDT

path_pdt = r"C:\Users\{CP}\SNCF\Pôle data - Lab' MTA GrpO365 - Documents\Projets\Hackathon\Données\Données brutes\Course_et_comptage_Ligne_H_2023.csv".format(CP=CP)
data_pdt = pd.read_csv(path_pdt, encoding='latin-1')
display(data_pdt.columns)


Index(['ï»¿', 'Ligne', 'AnnÃ©e', 'Date', 'Jour', 'NumÃ©ro_marche_realisÃ©e',
       'Gare', 'Rang_rÃ©alisÃ©', 'Origine', 'Terminus', 'Mission',
       'Heure_dÃ©part_thÃ©orique', 'Heure_dÃ©part_rÃ©alisÃ©',
       'Heure_arrivÃ©e_thÃ©orique', 'Heure_arrivÃ©e_rÃ©alisÃ©e',
       'Date_comptage', 'Charge', 'MontÃ©es', 'Descentes', 'CapacitÃ©_assise',
       'CapacitÃ©_totale', 'Taux_occupation_assis', 'Taux_occupation_debout',
       'Composition_thÃ©orique', 'Nombre_voitures_thÃ©orique',
       'Nombre_portes_thÃ©orique', 'Code_gare', 'Numero_marche_thÃ©orique',
       'Rang_thÃ©orique'],
      dtype='object')

In [6]:
data_pdt['Heure_dÃ©part_rÃ©alisÃ©'] = data_pdt['Heure_dÃ©part_rÃ©alisÃ©'].fillna(data_pdt['Heure_arrivÃ©e_rÃ©alisÃ©e'])
data_pdt['Jour-heure'] = pd.to_datetime(data_pdt['Heure_dÃ©part_rÃ©alisÃ©']).dt.round('5min').apply(lambda x:str(x)[:19])

data_pdt['tranche_horaire'] = data_pdt['Heure_dÃ©part_rÃ©alisÃ©'].apply(lambda x: int(str(x)[11:13])).replace({0:24,1:25,2:26})
data_pdt['demi_heure'] = data_pdt['Heure_dÃ©part_rÃ©alisÃ©'].apply(lambda x: "0-30" if int(str(x)[14:16])<30 else "30-60")


data_pdt['Date'] = data_pdt['Date'].apply(lambda x: str(x))
data_pdt_comptage = data_pdt.groupby(["Date",'Ligne',"Gare",'Code_gare','tranche_horaire','demi_heure','Jour-heure'],as_index=False).aggregate({'MontÃ©es':'sum', 'Taux_occupation_assis':'mean', 'Taux_occupation_debout':'mean'})
data_pdt_comptage = data_pdt_comptage.rename(columns={'MontÃ©es':'Montees','Taux_occupation_assis':'Taux_occupation_totale','Taux_occupation_debout':'Taux_occupation_assis','Code_gare':'Gare_A'})
display(data_pdt_comptage[(data_pdt_comptage.Date=="2023-11-01")&(data_pdt_comptage.Gare_A=="SDE")&(data_pdt_comptage.tranche_horaire==6)])
data_pdt_comptage = data_pdt_comptage.replace({"PNB":"GDS"})


,Date,Ligne,Gare,Gare_A,tranche_horaire,demi_heure,Jour-heure,Montees,Taux_occupation_totale,Taux_occupation_assis


In [52]:
path = r"C:\Users\{CP}\SNCF\Pôle data - Lab' MTA GrpO365 - Documents\Projets\Hackathon\Données\Données brutes\Base_de_donnees_finale_V3.csv".format(CP=CP)

df_final = pd.read_csv(path,encoding='latin-1',sep=',')
df_final['Jour-heure'] = df_final['Jour-heure'].astype(str)

#Merge avec les comptages
display(data_pdt_comptage[(data_pdt_comptage['Jour-heure']=="2023-11-01 04:10:00")&(data_pdt_comptage.Gare_A=="SDE")])
display(df_final[(df_final['Jour-heure']=="2023-11-01 06:20:00")&(df_final.Gare_A=="SDE")])
df_final = df_final.merge(data_pdt_comptage,on=['Gare_A','Jour-heure'],how='left').rename(columns={'Montees':"Montees_A","Taux_occupation_assis":"Taux_occ_assis_AB","Taux_occupation_totale":"Taux_occ_tot_AB"})
df_final = df_final.merge(data_pdt_comptage.rename(columns={"Gare_A":"Gare_B"}),on=['Gare_B','Jour-heure'],how='left').rename(columns={'Montees':"Montees_B"})

#Merge les temps d'attente
path_t = r"C:\Users\{CP}\SNCF\Pôle data - Lab' MTA GrpO365 - Documents\Projets\Hackathon\Données\Données pré-traitées\database_temps_attente.xlsx".format(CP=CP)
data_tattente = pd.read_excel(path_t)
data_tattente['Date'] = data_tattente['Date'].astype(str)
df_final = df_final.merge(data_tattente.rename(columns={'Gare':'Gare_A',"Date":"Jour-heure"}),on=['Gare_A','Jour-heure','Sens'],how='left').rename(columns={"temps_attente":"Temps_attente_A"})
df_final = df_final.merge(data_tattente.rename(columns={'Gare':'Gare_B',"Date":"Jour-heure"}),on=['Gare_B','Jour-heure','Sens'],how='left').rename(columns={"temps_attente":"Temps_attente_B"})

#Merge les validations

df_final['tranche_horaire'] = df_final['Jour-heure'].apply(lambda x: int(str(x)[11:13])).replace({0:24,1:25,2:26})
df_final['demi_heure'] = df_final['Jour-heure'].apply(lambda x: "0-30" if int(str(x)[14:16])<30 else "30-60")
df_final['Date'] = df_final['Jour-heure'].apply(lambda x: str(x)[:10])
data_val_final['Date'] = data_val_final['Date'].astype(str)

df_final = df_final[['Gare_A', 'Gare_B', 'Ordre',
       'Sens', 'Jour-heure', 'Trigramme_x', 'Lat_A', 'Long_A', 'Trigramme_y',
       'Lat_B', 'Long_B',  'Presence_gare_A', 'Presence_gare_B', 'Taux_criminalite_A', 'Taux_criminalite_B',
       'Ligne_y', 'Gare_x', 'tranche_horaire_x', 'demi_heure_x', 'Montees_A',
       'Taux_occ_tot_AB', 'Taux_occ_assis_AB', 'Montees_B',
       'Temps_attente_A', 'Temps_attente_B', 'tranche_horaire',
       'demi_heure', 'Date','Temps_A','Temps_B']]

df_final = df_final.merge(data_val_final,on=['Gare_A','tranche_horaire','demi_heure','Date'],how='left').rename(columns={"validations_entree_brutes":'Validations_A'})
df_final = df_final.merge(data_val_final.rename(columns={"Gare_A":"Gare_B"}),on=['Gare_B','tranche_horaire','demi_heure','Date'],how='left').rename(columns={"validations_entree_brutes":'Validations_B'})

df_final['Validations_A'] = df_final['Validations_A']/6
df_final['Validations_B'] = df_final['Validations_B']/6
df_final['Fraude_A'] = 1-df_final['Validations_A']/df_final['Montees_A']
df_final['Fraude_B'] = 1-df_final['Validations_B']/df_final['Montees_B']
df_final['Taux_occ_A'] = df_final['Validations_A']/600
df_final['Taux_occ_B'] = df_final['Validations_B']/600


df_final = df_final[['Gare_A', 'Gare_B', 'Ordre', 'Sens', 'Jour-heure', 'Trigramme_x',
       'Lat_A', 'Long_A', 'Trigramme_y', 'Lat_B', 'Long_B', 'Presence_gare_A',
       'Presence_gare_B', 'Taux_criminalite_A', 'Taux_criminalite_B', 'Montees_A',
       'Taux_occ_tot_AB', 'Taux_occ_assis_AB', 'Montees_B', 'Temps_attente_A',
       'Temps_attente_B', 'tranche_horaire', 'demi_heure', 'Date', 'Validations_A',
       'Validations_B', 'Fraude_A', 'Fraude_B', 'Taux_occ_A', 'Taux_occ_B','Temps_A','Temps_B','gare_x','gare_y']]
display(df_final)
#df_final.to_csv(r"C:\Users\{CP}\SNCF\Pôle data - Lab' MTA GrpO365 - Documents\Projets\Hackathon\Données\Données pré-traitées\Base_de_donnees_finale_merge.csv".format(CP=CP))


,Date,Ligne,Gare,Gare_A,tranche_horaire,demi_heure,Jour-heure,Montees,Taux_occupation_totale,Taux_occupation_assis
1072155,2023-11-01,H,St-Denis BÃ¢timent Voyageurs,SDE,4,0-30,2023-11-01 04:10:00,160.0,0.232646,0.454449


,Unnamed: 0.1,Unnamed: 0,Gare_A,Gare_B,Ligne,Ordre,Sens,Jour-heure,Trigramme_x,Lat_A,...,Temps_B,Presence_gare_A,Presence_gare_B,Fraude_A,Fraude_B,Taux_criminalite_A,Taux_criminalite_B,Indice_troncon_AB,Indice_gare_A,Indice_gare_B
4240,4240,4240,SDE,EPV,H,2,Vers Province,2023-11-01 06:20:00,SDE,48.935493,...,2,1.5,0.8,0.55,0.1,0.3,0.8,0,0,0
63544,63544,63544,SDE,EPV,H,13,Vers Paris,2023-11-01 06:20:00,SDE,48.935493,...,2,1.5,0.8,0.55,0.1,0.3,0.8,0,0,0


,Gare_A,Gare_B,Ordre,Sens,Jour-heure,Trigramme_x,Lat_A,Long_A,Trigramme_y,Lat_B,...,Validations_A,Validations_B,Fraude_A,Fraude_B,Taux_occ_A,Taux_occ_B,Temps_A,Temps_B,gare_x,gare_y
0,GDS,SDE,1,Vers Province,2023-11-01 06:00:00,NaN,NaN,NaN,SDE,48.935493,...,130.500000,29.333333,NaN,NaN,0.217500,0.048889,2,2,PARIS NORD,ST DENIS
1,GDS,SDE,1,Vers Province,2023-11-01 06:05:00,NaN,NaN,NaN,SDE,48.935493,...,130.500000,29.333333,NaN,NaN,0.217500,0.048889,2,2,PARIS NORD,ST DENIS
2,GDS,SDE,1,Vers Province,2023-11-01 06:10:00,NaN,NaN,NaN,SDE,48.935493,...,130.500000,29.333333,NaN,NaN,0.217500,0.048889,2,2,PARIS NORD,ST DENIS
3,GDS,SDE,1,Vers Province,2023-11-01 06:15:00,NaN,NaN,NaN,SDE,48.935493,...,130.500000,29.333333,NaN,NaN,0.217500,0.048889,2,2,PARIS NORD,ST DENIS
4,GDS,SDE,1,Vers Province,2023-11-01 06:20:00,NaN,NaN,NaN,SDE,48.935493,...,130.500000,29.333333,NaN,NaN,0.217500,0.048889,2,2,PARIS NORD,ST DENIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122383,SOL,GDS,3,Vers Paris,2023-11-15 22:35:00,SOL,49.033871,2.127934,NaN,NaN,...,0.333333,250.833333,NaN,NaN,0.000556,0.418056,2,2,ST OUEN LIESSE,PARIS NORD
122384,SOL,GDS,3,Vers Paris,2023-11-15 22:40:00,SOL,49.033871,2.127934,NaN,NaN,...,0.333333,250.833333,NaN,NaN,0.000556,0.418056,2,2,ST OUEN LIESSE,PARIS NORD
122385,SOL,GDS,3,Vers Paris,2023-11-15 22:45:00,SOL,49.033871,2.127934,NaN,NaN,...,0.333333,250.833333,NaN,NaN,0.000556,0.418056,2,2,ST OUEN LIESSE,PARIS NORD
122386,SOL,GDS,3,Vers Paris,2023-11-15 22:50:00,SOL,49.033871,2.127934,NaN,NaN,...,0.333333,250.833333,NaN,NaN,0.000556,0.418056,2,2,ST OUEN LIESSE,PARIS NORD


In [53]:
display(df_final.Taux_occ_assis_AB.unique())
display(df_final.Taux_occ_assis_AB.min())

array([       nan, 0.32521186, 0.30508475, ..., 0.30826271, 0.13294492,
       0.24841102])

0.0

In [54]:
def score_occupation_bord(taux_occ_bord):
    taux_occ_bord[(taux_occ_bord >= 0.5)] = 3
    taux_occ_bord[(taux_occ_bord >= 0.25) & (taux_occ_bord < 0.5)] = 2
    taux_occ_bord[(taux_occ_bord >= 0.1) & (taux_occ_bord < 0.25)] = 1
    taux_occ_bord[taux_occ_bord < 0.1] = 4
    display(taux_occ_bord.unique())
    return(taux_occ_bord)
    
def score_occupation_quai(taux_occ_quai):
    taux_occ_quai[(taux_occ_quai >= 0.5)] = 3
    taux_occ_quai[(taux_occ_quai >= 0.25) & (taux_occ_quai < 0.5)] = 2
    taux_occ_quai[(taux_occ_quai >= 0.1) & (taux_occ_quai < 0.25)] = 1
    taux_occ_quai[taux_occ_quai < 0.1] = 4
    
    return(taux_occ_quai)
    
def score_temps_trajet(temps_trajet):
    temps_trajet = 1
    return(temps_trajet)
    
def score_temps_attente(temps_attente):
    temps_attente[temps_attente < 2] = 1
    temps_attente[(temps_attente >= 2) & (temps_attente < 5)] = 2
    temps_attente[(temps_attente >= 5)] = 3
    return(temps_attente)

def score_presence_en_gare(presence_en_gare):
    presence_en_gare = 1
    return(presence_en_gare)

def score_fraude(fraude):
    fraude[fraude >= 0.5] = 4
    fraude[(fraude >= 0.2) & (fraude < 0.5) ] = 3
    fraude[(fraude >= 0.1) & (fraude < 0.2) ] = 2
    fraude[(fraude < 0.1) ] = 1
    return(fraude)

def score_taux_criminalite(taux_criminalite):
    taux_criminalite = 1
    return(taux_criminalite)
    
def feel_safe_score(data):
    # Taux d'occupation
    score_occupation_A = score_occupation_quai(data["Taux_occ_A"]) 
    score_occupation_B = score_occupation_quai(data["Taux_occ_B"]) 
    score_occupation_AB = score_occupation_bord(data["Taux_occ_assis_AB"]) 
    
    # Temps de contact avec le service
    # Temps d'attente
    score_attente_A = score_temps_attente(data["Temps_A"])
    score_attente_B = score_temps_attente(data["Temps_B"])
     
    # score_attente_AB = ...
    
    # Fraude
   
    score_fraude_A = score_fraude(data["Fraude_A"])
    score_fraude_B = score_fraude(data["Fraude_B"]) 
    
    # Présence en gare
    score_presence_gare_A = score_presence_en_gare(data["Presence_gare_A"]) 
    score_presence_gare_B = score_presence_en_gare(data["Presence_gare_B"]) 
  
    # Taux de criminalité
    score_criminalite_gare_A = score_presence_en_gare(data["Taux_criminalite_A"]) 
    score_criminalite_gare_B = score_presence_en_gare(data["Taux_criminalite_B"])
    
    max_score_A = 4 * 3 + 3 * 1 + 3
    max_score_B = 4 * 3 + 3 * 1 + 3
    max_score_AB = 4
    
    
    data["Indice_troncon_AB"] = score_occupation_AB / max_score_AB
    data["Indice_gare_A"] = (score_occupation_A + score_attente_A + score_fraude_A +
                                   score_presence_gare_A + score_criminalite_gare_A) / max_score_A
    data["Indice_gare_B"] = (score_occupation_B + score_attente_B + score_fraude_B +
                                   score_presence_gare_B + score_criminalite_gare_B) / max_score_B
    
    return data
    

In [55]:
data_final_score = feel_safe_score(df_final)
data_final_score = data_final_score[(data_final_score.tranche_horaire>=6) & (data_final_score.tranche_horaire<=22)]
display(data_final_score.Fraude_B.unique())
data_final_score.to_csv(r"C:\Users\{CP}\SNCF\Pôle data - Lab' MTA GrpO365 - Documents\Projets\Hackathon\Données\Données pré-traitées\Base_de_donnees_finale.csv".format(CP=CP))


C:\Users\9920510N\AppData\Local\Temp\ipykernel_34188\2935201899.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taux_occ_quai[(taux_occ_quai >= 0.25) & (taux_occ_quai < 0.5)] = 2
C:\Users\9920510N\AppData\Local\Temp\ipykernel_34188\2935201899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taux_occ_quai[(taux_occ_quai >= 0.1) & (taux_occ_quai < 0.25)] = 1
C:\Users\9920510N\AppData\Local\Temp\ipykernel_34188\2935201899.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

array([nan,  2.,  4.,  1.,  3.])

C:\Users\9920510N\AppData\Local\Temp\ipykernel_34188\2935201899.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps_attente[(temps_attente >= 2) & (temps_attente < 5)] = 2
C:\Users\9920510N\AppData\Local\Temp\ipykernel_34188\2935201899.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps_attente[(temps_attente >= 5)] = 3
C:\Users\9920510N\AppData\Local\Temp\ipykernel_34188\2935201899.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

array([nan,  1.,  2.,  3.,  4.])